In [ ]:
def bronze_budget():

    # This dictionary contains the TRUTH for project country and currency,
    # overriding any inconsistencies found in source data.
    PROJECT_CURRENCY_MAP = {
        "BE01": {"country": "Belgium", "currency": "EUR"},
        "BE55": {"country": "Belgium", "currency": "EUR"},
        "KE01": {"country": "Kenya", "currency": "KES"},   # Corrected: Force KES for KE01 expenses, overriding DB's XOF
        "KEO2": {"country": "Kenya", "currency": "KES"},
        "SN01": {"country": "Senegal", "currency": "XOF"}, # Corrected: Force Senegal for SN01 country, overriding DB's Kenya
        "SN02": {"country": "Senegal", "currency": "XOF"},
        "BF01": {"country": "Burkina Faso", "currency": "XOF"},
        "BF02": {"country": "Burkina Faso", "currency": "XOF"},
    }

    DATA_DIRECTORY = "/Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025"
    OUTPUT_DIRECTORY = "/Users/hajirufai/test/MSF-test/processed_data"

    # Read csvs (budget data)
    import pandas as pd
    import os

    # --- Function to extract budget from a single CSV ---
    def extract_budget_from_csv(csv_path: str) -> pd.DataFrame:
        """
        Reads budget data from a CSV file.
        """
        try:
            df = pd.read_csv(csv_path)
            return df
        except Exception as e:
            print(f"Error reading budget from {csv_path}: {e}")
            return pd.DataFrame()
        
    # --- Main logic to iterate and combine CSVs ---
    all_budget_df = pd.DataFrame()
    csv_files_found = 0

    print(f"\n--- Bronze Layer: Starting CSV Data Ingestion from {DATA_DIRECTORY} ---")

    if not os.path.exists(DATA_DIRECTORY):
        print(f"Error: Data directory '{DATA_DIRECTORY}' not found. Please verify the path.")
    else:
        for filename in os.listdir(DATA_DIRECTORY):
            file_path = os.path.join(DATA_DIRECTORY, filename)
            
            # Process only files ending with '_budget.csv' and skip directories
            if os.path.isdir(file_path) or not filename.endswith("_budget.csv"):
                continue

            # Derive project_id from filename (e.g., "BE01" from "BE01_budget.csv")
            project_name_from_file = os.path.splitext(filename)[0]
            print(f"project_name_from_file: {project_name_from_file}")
            base_project_id = project_name_from_file.replace('_budget', '')

            # Get country from our PROJECT_CURRENCY_MAP
            project_info = PROJECT_CURRENCY_MAP.get(base_project_id)
            if not project_info:
                print(f"Warning: Project '{base_project_id}' (from '{filename}') not found in PROJECT_CURRENCY_MAP. Skipping.")
                continue
            
            country_from_map = project_info['country']

            print(f"  Ingesting budget from CSV: {filename} (Project: {base_project_id}, Country: {country_from_map})...")
            df = extract_budget_from_csv(file_path)
            # print(f"df: \n{df.head()}")
            
            if not df.empty:
                df['project_id'] = base_project_id
                df['country'] = country_from_map
                # Also add the original_currency_map for consistency, even if budget is already in EUR
                all_budget_df = pd.concat([all_budget_df, df], ignore_index=True)
                csv_files_found += 1
            else:
                print(f" No data extracted or error occurred for {filename}.")

    return all_budget_df


In [ ]:
budget_df = bronze_budget()


In [ ]:
budget_df.head()


In [ ]:
import pandas as pd
import sqlite3
import os

DATA_DIRECTORY = "/Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025"
OUTPUT_DIRECTORY = "/Users/hajirufai/test/MSF-test/processed_data"

def bronze_expenses()-> pd.DataFrame:
    # --- Bronze Layer: Ingesting Expenses from SQLite DBs ---
    def extract_expenses_from_db(db_path: str) -> pd.DataFrame:
        """
        Connects to a SQLite database and extracts all data from the 'expenses' table.
        """
        conn = None
        try:
            conn = sqlite3.connect(db_path)
            query = "SELECT * FROM expenses" # Assumes 'expenses' is the table name
            df = pd.read_sql_query(query, conn)
            return df
        except sqlite3.Error as e:
            print(f"Error reading expenses from {db_path}: {e}")
            return pd.DataFrame() # Return empty DataFrame on error
        finally:
            if conn:
                conn.close()


    # --- Main logic to iterate and combine DBs ---
    all_expenses_df = pd.DataFrame()
    db_files_found = 0

    # Define project currency map for expenses
    PROJECT_CURRENCY_MAP = {
        "BE01": {"country": "Belgium", "currency": "EUR"},
        "BE55": {"country": "Belgium", "currency": "EUR"},
        "KE01": {"country": "Kenya", "currency": "KES"},
        "KE02": {"country": "Kenya", "currency": "KES"},   
        "SN01": {"country": "Senegal", "currency": "XOF"},
        "SN02": {"country": "Senegal", "currency": "XOF"},
        "BF01": {"country": "Burkina Faso", "currency": "XOF"},
        "BF02": {"country": "Burkina Faso", "currency": "XOF"},
    }

    print(f"\n--- Bronze Layer: Starting DB Data Ingestion from {DATA_DIRECTORY} ---")

    if not os.path.exists(DATA_DIRECTORY):
        print(f"Error: Data directory '{DATA_DIRECTORY}' not found. Please verify the path.")
    else:
        for filename in os.listdir(DATA_DIRECTORY):
            file_path = os.path.join(DATA_DIRECTORY, filename)
            
            # Process only files ending with '.db' and skip directories
            if os.path.isdir(file_path) or not filename.endswith(".db"):
                continue

            # Derive project_id from filename (e.g., "BE01" from "BE01.db")
            project_name_from_file = os.path.splitext(filename)[0]
            base_project_id = project_name_from_file # For DBs, the filename directly is the project_id

            # Get country and currency from our PROJECT_CURRENCY_MAP
            project_info = PROJECT_CURRENCY_MAP.get(base_project_id)
            if not project_info:
                print(f"Warning: Project '{base_project_id}' (from '{filename}') not found in PROJECT_CURRENCY_MAP. Skipping.")
                continue
            
            country_from_map = project_info['country']
            currency_from_map = project_info['currency'] # This will be our source of truth for currency

            print(f"  Ingesting expenses from DB: {filename} (Project: {base_project_id}, Country: {country_from_map})...")
            df = extract_expenses_from_db(file_path)
            
            if not df.empty:
                df['project_id'] = base_project_id
                df['country'] = country_from_map
                df['original_currency'] = currency_from_map # Add the true currency from our map
                all_expenses_df = pd.concat([all_expenses_df, df], ignore_index=True)
                db_files_found += 1
            else:
                print(f"   No data extracted or error occurred for {filename}.")

    return all_expenses_df


In [ ]:
df = bronze_expenses()

In [ ]:

df.head()


In [ ]:
print("Hello")

In [ ]:
# USe the bronze_layer to get expenses df and budget df
import pandas as pd
import sqlite3
import os
from bronze_layer import bronze_expenses, bronze_budget

DATA_DIRECTORY = "/Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025"
OUTPUT_DIRECTORY = "/Users/hajirufai/test/MSF-test/processed_data"

expenses_df = bronze_expenses()
budget_df = bronze_budget()

In [ ]:
display(expenses_df.head())
budget_df.head()

In [ ]:
# USe the bronze_layer to get expenses df and budget df
import pandas as pd
import sqlite3
import os
from bronze_layer import bronze_expenses, bronze_budget

DATA_DIRECTORY = "/Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025"
OUTPUT_DIRECTORY = "/Users/hajirufai/test/MSF-test/processed_data"

expenses_df = bronze_expenses()
budget_df = bronze_budget()

In [ ]:
expenses_df.info()

In [ ]:
budget_df.info()

In [ ]:
# modify KEO1 to KE01 in bronze budget df
budget_df['project_id'] = budget_df['project_id'].replace('KEO1', 'KE01')
budget_df.info()

In [ ]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd

In [ ]:
# Create a date column:
# convert the 'year' and 'month' columns to string format
budget_df['year'] = budget_df['year'].astype(str)
budget_df['month'] = budget_df['month'].astype(str)

# create a temporary date string with the 1st day of the month
budget_df['temp_date_str'] = budget_df['year'] + '-' + budget_df['month'].str.zfill(2) + '-01'

# convert the temporary date string to datetime objects
budget_df['date'] = pd.to_datetime(budget_df['temp_date_str'], errors='coerce')

# convert to the last day of the month using MonthEnd
budget_df['date'] = budget_df['date'] + MonthEnd(1)

# drop the intermediate columns if no longer needed
budget_df = budget_df.drop(columns=['year', 'month', 'temp_date_str'])

In [ ]:
budget_df.info()

## RESTART

In [1]:
import pandas as pd
import sqlite3
import os

DATA_DIRECTORY = "/Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025"
OUTPUT_DIRECTORY = "/Users/hajirufai/test/MSF-test/processed_data"

def bronze_budget()-> pd.DataFrame:
    """Returns a dataframe containing all budget data from all CSVs."""

    # This dictionary contains the TRUTH for project country and currency,
    # overriding any inconsistencies found in source data.
    PROJECT_CURRENCY_MAP = {
        "BE01": {"country": "Belgium", "currency": "EUR"},
        "BE55": {"country": "Belgium", "currency": "EUR"},
        "KE01": {"country": "Kenya", "currency": "KES"},   # Corrected: Force KES for KE01 expenses, overriding DB's XOF
        "KEO2": {"country": "Kenya", "currency": "KES"},
        "SN01": {"country": "Senegal", "currency": "XOF"}, # Corrected: Force Senegal for SN01 country, overriding DB's Kenya
        "SN02": {"country": "Senegal", "currency": "XOF"},
        "BF01": {"country": "Burkina Faso", "currency": "XOF"},
        "BF02": {"country": "Burkina Faso", "currency": "XOF"},
    }

    DATA_DIRECTORY = "/Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025"
    OUTPUT_DIRECTORY = "/Users/hajirufai/test/MSF-test/processed_data"

    # Read csvs (budget data)
    import pandas as pd
    import os

    # --- Function to extract budget from a single CSV ---
    def extract_budget_from_csv(csv_path: str) -> pd.DataFrame:
        """
        Reads budget data from a CSV file.
        """
        try:
            df = pd.read_csv(csv_path)
            return df
        except Exception as e:
            print(f"Error reading budget from {csv_path}: {e}")
            return pd.DataFrame()
        
    # --- Main logic to iterate and combine CSVs ---
    all_budget_df = pd.DataFrame()
    csv_files_found = 0

    print(f"\n--- Bronze Layer: Starting CSV Data Ingestion from {DATA_DIRECTORY} ---")

    if not os.path.exists(DATA_DIRECTORY):
        print(f"Error: Data directory '{DATA_DIRECTORY}' not found. Please verify the path.")
    else:
        for filename in os.listdir(DATA_DIRECTORY):
            file_path = os.path.join(DATA_DIRECTORY, filename)
            
            # Process only files ending with '_budget.csv' and skip directories
            if os.path.isdir(file_path) or not filename.endswith("_budget.csv"):
                continue

            # Derive project_id from filename (e.g., "BE01" from "BE01_budget.csv")
            project_name_from_file = os.path.splitext(filename)[0]
            print(f"project_name_from_file: {project_name_from_file}")
            base_project_id = project_name_from_file.replace('_budget', '')

            # Get country from our PROJECT_CURRENCY_MAP
            project_info = PROJECT_CURRENCY_MAP.get(base_project_id)
            if not project_info:
                print(f"Warning: Project '{base_project_id}' (from '{filename}') not found in PROJECT_CURRENCY_MAP. Skipping.")
                continue
            
            country_from_map = project_info['country']

            print(f"  Ingesting budget from CSV: {filename} (Project: {base_project_id}, Country: {country_from_map})...")
            df = extract_budget_from_csv(file_path)
            # print(f"df: \n{df.head()}")
            
            if not df.empty:
                df['project_id'] = base_project_id
                df['country'] = country_from_map
                # Also add the original_currency_map for consistency, even if budget is already in EUR
                all_budget_df = pd.concat([all_budget_df, df], ignore_index=True)
                csv_files_found += 1
            else:
                print(f" No data extracted or error occurred for {filename}.")

    return all_budget_df

def bronze_expenses()-> pd.DataFrame:
    """Returns a DataFrame containing all expenses from all DBs."""
    # --- Bronze Layer: Ingesting Expenses from SQLite DBs ---
    def extract_expenses_from_db(db_path: str) -> pd.DataFrame:
        """
        Connects to a SQLite database and extracts all data from the 'expenses' table.
        """
        conn = None
        try:
            conn = sqlite3.connect(db_path)
            query = "SELECT * FROM expenses" # Assumes 'expenses' is the table name
            df = pd.read_sql_query(query, conn)
            return df
        except sqlite3.Error as e:
            print(f"Error reading expenses from {db_path}: {e}")
            return pd.DataFrame() # Return empty DataFrame on error
        finally:
            if conn:
                conn.close()


    # --- Main logic to iterate and combine DBs ---
    all_expenses_df = pd.DataFrame()
    db_files_found = 0

    # Define project currency map for expenses
    PROJECT_CURRENCY_MAP = {
        "BE01": {"country": "Belgium", "currency": "EUR"},
        "BE55": {"country": "Belgium", "currency": "EUR"},
        "KE01": {"country": "Kenya", "currency": "KES"},
        "KE02": {"country": "Kenya", "currency": "KES"},   
        "SN01": {"country": "Senegal", "currency": "XOF"},
        "SN02": {"country": "Senegal", "currency": "XOF"},
        "BF01": {"country": "Burkina Faso", "currency": "XOF"},
        "BF02": {"country": "Burkina Faso", "currency": "XOF"},
    }

    print(f"\n--- Bronze Layer: Starting DB Data Ingestion from {DATA_DIRECTORY} ---")

    if not os.path.exists(DATA_DIRECTORY):
        print(f"Error: Data directory '{DATA_DIRECTORY}' not found. Please verify the path.")
    else:
        for filename in os.listdir(DATA_DIRECTORY):
            file_path = os.path.join(DATA_DIRECTORY, filename)
            
            # Process only files ending with '.db' and skip directories
            if os.path.isdir(file_path) or not filename.endswith(".db"):
                continue

            # Derive project_id from filename (e.g., "BE01" from "BE01.db")
            project_name_from_file = os.path.splitext(filename)[0]
            base_project_id = project_name_from_file # For DBs, the filename directly is the project_id

            # Get country and currency from our PROJECT_CURRENCY_MAP
            project_info = PROJECT_CURRENCY_MAP.get(base_project_id)
            if not project_info:
                print(f"Warning: Project '{base_project_id}' (from '{filename}') not found in PROJECT_CURRENCY_MAP. Skipping.")
                continue
            
            country_from_map = project_info['country']
            currency_from_map = project_info['currency'] # This will be our source of truth for currency

            print(f"  Ingesting expenses from DB: {filename} (Project: {base_project_id}, Country: {country_from_map})...")
            df = extract_expenses_from_db(file_path)
            
            if not df.empty:
                df['project_id'] = base_project_id
                df['country'] = country_from_map
                df['original_currency'] = currency_from_map # Add the true currency from our map
                all_expenses_df = pd.concat([all_expenses_df, df], ignore_index=True)
                db_files_found += 1
            else:
                print(f"   No data extracted or error occurred for {filename}.")

    return all_expenses_df

In [4]:
# USe the bronze_layer to get expenses df and budget df
import pandas as pd
import sqlite3
import os
from bronze_layer import bronze_expenses, bronze_budget
from pandas.tseries.offsets import MonthEnd

DATA_DIRECTORY = "/Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025"
OUTPUT_DIRECTORY = "/Users/hajirufai/test/MSF-test/processed_data"


In [5]:
def silver_budget() -> pd.DataFrame:
    """Returns a clean budget dataframe."""
    budget_df = bronze_budget()
    # modify KEO1 to KE01 in bronze budget df
    budget_df['project_id'] = budget_df['project_id'].replace('KEO1', 'KE01')

    # drop version column, id column
    budget_df.drop(columns=['version', 'id'], inplace=True)

    # Create a date column:
    # convert the 'year' and 'month' columns to string format
    budget_df['year'] = budget_df['year'].astype(str)
    budget_df['month'] = budget_df['month'].astype(str)

    # create a temporary date string with the 1st day of the month
    budget_df['temp_date_str'] = budget_df['year'] + '-' + budget_df['month'].str.zfill(2) + '-01'

    # convert the temporary date string to datetime objects
    budget_df['date'] = pd.to_datetime(budget_df['temp_date_str'], errors='coerce')

    # convert to the last day of the month using MonthEnd
    budget_df['date'] = budget_df['date'] + MonthEnd(1)

    # drop the intermediate columns 
    budget_df = budget_df.drop(columns=['year', 'month', 'temp_date_str'])

    return budget_df

In [6]:
silver_budget_df = silver_budget()


--- Bronze Layer: Starting CSV Data Ingestion from /Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025 ---
project_name_from_file: BE55_budget
  Ingesting budget from CSV: BE55_budget.csv (Project: BE55, Country: Belgium)...
project_name_from_file: SN02_budget
  Ingesting budget from CSV: SN02_budget.csv (Project: SN02, Country: Senegal)...
project_name_from_file: BF01_budget
  Ingesting budget from CSV: BF01_budget.csv (Project: BF01, Country: Burkina Faso)...
project_name_from_file: BE01_budget
  Ingesting budget from CSV: BE01_budget.csv (Project: BE01, Country: Belgium)...
project_name_from_file: BF02_budget
  Ingesting budget from CSV: BF02_budget.csv (Project: BF02, Country: Burkina Faso)...
project_name_from_file: SN01_budget
  Ingesting budget from CSV: SN01_budget.csv (Project: SN01, Country: Senegal)...
project_name_from_file: KE01_budget
  Ingesting budget from CSV: KE01_budget.csv (Project: KE01, Country: Kenya)...
project_name_from_file: KEO2_budget
  Ingesting budget from

In [7]:
silver_budget_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   department  3456 non-null   object        
 1   category    3456 non-null   object        
 2   budget_eur  3456 non-null   float64       
 3   project_id  3456 non-null   object        
 4   country     3456 non-null   object        
 5   date        3456 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 162.1+ KB


In [8]:
silver_budget_df.isnull().sum()

department    0
category      0
budget_eur    0
project_id    0
country       0
date          0
dtype: int64

In [9]:
# Manipulating to get silver_expenses_df
expenses_df = bronze_expenses()

expenses_df.info()


--- Bronze Layer: Starting DB Data Ingestion from /Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025 ---
  Ingesting expenses from DB: BE55.db (Project: BE55, Country: Belgium)...
  Ingesting expenses from DB: BE01.db (Project: BE01, Country: Belgium)...
  Ingesting expenses from DB: BF02.db (Project: BF02, Country: Burkina Faso)...
  Ingesting expenses from DB: KE02.db (Project: KE02, Country: Kenya)...
  Ingesting expenses from DB: SN02.db (Project: SN02, Country: Senegal)...
  Ingesting expenses from DB: SN01.db (Project: SN01, Country: Senegal)...
  Ingesting expenses from DB: KE01.db (Project: KE01, Country: Kenya)...
  Ingesting expenses from DB: BF01.db (Project: BF01, Country: Burkina Faso)...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3456 non-null   int64  
 1   year               3456 non-null

In [ ]:
# Drop id and currency columns
expenses_df.drop(columns=['id', 'currency'], inplace=True)

# Create a date column:
# convert the 'year' and 'month' columns to string format
expenses_df['year'] = expenses_df['year'].astype(str)
expenses_df['month'] = expenses_df['month'].astype(str)

# create a temporary date string with the 1st day of the month
expenses_df['temp_date_str'] = expenses_df['year'] + '-' + expenses_df['month'].str.zfill(2) + '-01'

# convert the temporary date string to datetime objects
expenses_df['date'] = pd.to_datetime(expenses_df['temp_date_str'], errors='coerce')

# convert to the last day of the month using MonthEnd
expenses_df['date'] = expenses_df['date'] + MonthEnd(1)

# drop the intermediate columns 
expenses_df = expenses_df.drop(columns=['year', 'month', 'temp_date_str'])

# create rate column



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   department         3456 non-null   object        
 1   category           3456 non-null   object        
 2   amount_local       2880 non-null   float64       
 3   project_id         3456 non-null   object        
 4   country            3456 non-null   object        
 5   original_currency  3456 non-null   object        
 6   amount_eur         432 non-null    float64       
 7   date               3456 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 216.1+ KB


In [ ]:
# Drop amount_eur column
expenses_df.drop(columns=['amount_eur'], inplace=True)

expenses_df.info()

In [11]:
# Define our rate map
from get_latest_exchange_rate import get_latest_exchange_rate

eur_to_eur_rate = 1.0 # EUR to EUR conversion rate is 1
kes_to_eur_rate = get_latest_exchange_rate("KES", "EUR")
xof_to_eur_rate = get_latest_exchange_rate("XOF", "EUR")

rate_mapping = {
    'EUR': eur_to_eur_rate,
    'KES': kes_to_eur_rate,
    'XOF': xof_to_eur_rate
}

# Add rate column
expenses_df['rate'] = expenses_df['original_currency'].map(rate_mapping)

# Add amount_eur column
expenses_df['amount_eur'] = expenses_df['amount_local'] * expenses_df['rate']


In [12]:
expenses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   department         3456 non-null   object        
 1   category           3456 non-null   object        
 2   amount_local       2880 non-null   float64       
 3   project_id         3456 non-null   object        
 4   country            3456 non-null   object        
 5   original_currency  3456 non-null   object        
 6   amount_eur         2880 non-null   float64       
 7   date               3456 non-null   datetime64[ns]
 8   rate               3456 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 243.1+ KB


In [15]:
expenses_df["amount_eur"].isnull().sum()

np.int64(576)

In [17]:
silver_expenses_df = expenses_df

In [18]:
silver_expenses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   department         3456 non-null   object        
 1   category           3456 non-null   object        
 2   amount_local       2880 non-null   float64       
 3   project_id         3456 non-null   object        
 4   country            3456 non-null   object        
 5   original_currency  3456 non-null   object        
 6   amount_eur         2880 non-null   float64       
 7   date               3456 non-null   datetime64[ns]
 8   rate               3456 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 243.1+ KB


# GOLD LAYER

In [2]:
import numpy as np
import pandas as pd
import os
from silver_layer import silver_budget, silver_expenses

# def gold():
#     """Returns a set of gold dataframes from the silver dataframes"""
#     silver_budget_df = silver_budget()
#     silver_expenses_df = silver_expenses()

In [3]:
silver_budget_df = silver_budget()
silver_expenses_df = silver_expenses()


--- Bronze Layer: Starting CSV Data Ingestion from /Users/hajirufai/test/MSF-test/OneDrive_1_7-4-2025 ---
project_name_from_file: BE55_budget
  Ingesting budget from CSV: BE55_budget.csv (Project: BE55, Country: Belgium)...
project_name_from_file: SN02_budget
  Ingesting budget from CSV: SN02_budget.csv (Project: SN02, Country: Senegal)...
project_name_from_file: BF01_budget
  Ingesting budget from CSV: BF01_budget.csv (Project: BF01, Country: Burkina Faso)...
project_name_from_file: BE01_budget
  Ingesting budget from CSV: BE01_budget.csv (Project: BE01, Country: Belgium)...
project_name_from_file: BF02_budget
  Ingesting budget from CSV: BF02_budget.csv (Project: BF02, Country: Burkina Faso)...
project_name_from_file: SN01_budget
  Ingesting budget from CSV: SN01_budget.csv (Project: SN01, Country: Senegal)...
project_name_from_file: KE01_budget
  Ingesting budget from CSV: KE01_budget.csv (Project: KE01, Country: Kenya)...
project_name_from_file: KEO2_budget
  Ingesting budget from

In [22]:
silver_budget_df.info()
silver_expenses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   department  3456 non-null   object        
 1   category    3456 non-null   object        
 2   budget_eur  3456 non-null   float64       
 3   project_id  3456 non-null   object        
 4   country     3456 non-null   object        
 5   date        3456 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 162.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   department         3456 non-null   object        
 1   category           3456 non-null   object        
 2   amount_local       2880 non-null   float64       
 3   project_id         3456 non-null   object        
 

In [5]:
silver_budget_df['project_id'].unique()

array(['BE55', 'SN02', 'BF01', 'BE01', 'BF02', 'SN01', 'KE01', 'KE02'],
      dtype=object)

In [6]:
# check if set of project id in expenses == set of project id in budget
print(set(silver_expenses_df['project_id'].unique()) == set(silver_budget_df['project_id'].unique()))

True


In [8]:
# # correct the issue above
# # inspect first
# print(set(silver_expenses_df['project_id'].unique()))
# print(set(silver_budget_df['project_id'].unique()))

In [9]:
# combine the silver_budget_df and silver_expenses_df
# with common columns: project_id, date, country, department, category

In [10]:
# Merge the silver_budget_df and silver_expenses_df
common_columns = ['date', 'project_id', 'country', 'department', 'category']

merged_df = pd.merge(silver_budget_df, silver_expenses_df, on=common_columns, how='inner')  # inner is default
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   department         3456 non-null   object        
 1   category           3456 non-null   object        
 2   budget_eur         3456 non-null   float64       
 3   project_id         3456 non-null   object        
 4   country            3456 non-null   object        
 5   date               3456 non-null   datetime64[ns]
 6   amount_local       2880 non-null   float64       
 7   original_currency  3456 non-null   object        
 8   rate               3456 non-null   float64       
 9   amount_eur         2880 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 270.1+ KB


In [11]:
# Create a snowflake schema from the merged_df

In [12]:
# DimDepartment
# Extract unique departments and create a surrogate key
dim_department = merged_df[['department']].drop_duplicates().reset_index(drop=True)
dim_department['department_key'] = dim_department.index + 1 # Start keys from 1
dim_department = dim_department[['department_key', 'department']] # Reorder columns
dim_department.rename(columns={'department': 'department_name'}, inplace=True)
print("DimDepartment head:")
print(dim_department.head())
print("\n")

DimDepartment head:
   department_key department_name
0               1              HR
1               2         Medical
2               3       Logistics
3               4          Supply




In [13]:
# DimCategory
dim_category = merged_df[['category']].drop_duplicates().reset_index(drop=True)
dim_category['category_key'] = dim_category.index + 1
dim_category = dim_category[['category_key', 'category']]
dim_category.rename(columns={'category': 'category_name'}, inplace=True)
print("DimCategory head:")
print(dim_category.head())
print("\n")

DimCategory head:
   category_key      category_name
0             1           Salaries
1             2           Training
2             3        Recruitment
3             4        Medications
4             5  Medical Equipment




In [14]:
# DimProject
dim_project = merged_df[['project_id']].drop_duplicates().reset_index(drop=True)
dim_project['project_key'] = dim_project.index + 1
dim_project = dim_project[['project_key', 'project_id']]
dim_project.rename(columns={'project_id': 'project_name'}, inplace=True) # Assuming project_id is also the name
print("DimProject head:")
print(dim_project.head())
print("\n")

DimProject head:
   project_key project_name
0            1         BE55
1            2         SN02
2            3         BF01
3            4         BE01
4            5         BF02




In [16]:
# convert the merged df to csv
merged_df.to_csv('/Users/hajirufai/test/MSF-test2/processed_data/merged_df.csv', index=False)

In [17]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   department         3456 non-null   object        
 1   category           3456 non-null   object        
 2   budget_eur         3456 non-null   float64       
 3   project_id         3456 non-null   object        
 4   country            3456 non-null   object        
 5   date               3456 non-null   datetime64[ns]
 6   amount_local       2880 non-null   float64       
 7   original_currency  3456 non-null   object        
 8   rate               3456 non-null   float64       
 9   amount_eur         2880 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 270.1+ KB


In [18]:
df = pd.read_csv('/Users/hajirufai/test/MSF-test2/processed_data/merged_df.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   department         3456 non-null   object 
 1   category           3456 non-null   object 
 2   budget_eur         3456 non-null   float64
 3   project_id         3456 non-null   object 
 4   country            3456 non-null   object 
 5   date               3456 non-null   object 
 6   amount_local       2880 non-null   float64
 7   original_currency  3456 non-null   object 
 8   rate               3456 non-null   float64
 9   amount_eur         2880 non-null   float64
dtypes: float64(4), object(6)
memory usage: 270.1+ KB


In [19]:
df['department'].unique()

array(['HR', 'Medical', 'Logistics', 'Supply'], dtype=object)

In [20]:
df['category'].unique()

array(['Salaries', 'Training', 'Recruitment', 'Medications',
       'Medical Equipment', 'Consultations', 'Fuel', 'Transport',
       'Vehicle Maintenance', 'Supplies', 'Warehousing', 'Cold Chain'],
      dtype=object)

In [22]:
df['category'].nunique()

12

In [23]:
df['date'].nunique()

36

In [24]:
df['country'].unique()  

array(['Belgium', 'Senegal', 'Burkina Faso', 'Kenya'], dtype=object)

In [25]:
df['project_id'].unique()

array(['BE55', 'SN02', 'BF01', 'BE01', 'BF02', 'SN01', 'KE01', 'KE02'],
      dtype=object)

In [29]:
df['rate'].nunique()

3

In [30]:
df['amount_local'].nunique()

2878

# LAST pARt

### Fact & Dim Table Modeling in Python

In [5]:
import pandas as pd

# Load your final Gold layer data
df = pd.read_csv('gold_df.csv')

# Convert 'date' to YYYYMMDD integer format
df['date_id'] = pd.to_datetime(df['date']).dt.strftime('%Y%m%d').astype(int)

# Build dimension tables
dim_date = df[['date_id', 'date']].drop_duplicates().reset_index(drop=True)

dim_department = df[['department']].drop_duplicates().reset_index(drop=True)
dim_department['department_id'] = dim_department.index + 1

dim_country = df[['country']].drop_duplicates().reset_index(drop=True)
dim_country['country_id'] = dim_country.index + 1

dim_category = df[['category']].drop_duplicates().reset_index(drop=True)
dim_category['category_id'] = dim_category.index + 1

dim_project = df[['project_id']].drop_duplicates().reset_index(drop=True)
dim_project['project_id_numeric'] = dim_project.index + 1

# Merge keys back into the main dataframe
df = df.merge(dim_department, on='department', how='left') \
       .merge(dim_country, on='country', how='left') \
       .merge(dim_category, on='category', how='left') \
       .merge(dim_project, on='project_id', how='left')

# Fact Table
fact_expenses = df[['date_id', 'department_id', 'country_id', 'category_id', 'project_id_numeric', 'budget_eur', 'amount_eur']]

# Save to CSVs
dim_date.to_csv('dim_date.csv', index=False)
dim_department.to_csv('dim_department.csv', index=False)
dim_country.to_csv('dim_country.csv', index=False)
dim_category.to_csv('dim_category.csv', index=False)
dim_project.to_csv('dim_project.csv', index=False)
fact_expenses.to_csv('fact_expenses.csv', index=False)
